In [156]:
import praw
import pprint
import textwrap
import json

from math import sqrt, log
from hashlib import sha1
from graphviz import Digraph
from credentials import ID, SECRET
from praw.models import MoreComments as more

In [157]:
reddit = praw.Reddit(
    client_id=ID,
    client_secret=SECRET,
    user_agent="The Rhetor Project"
)

ID = "17llow9"
post = reddit.submission(id=ID)
topic = post.title[5:]
desc = post.selftext
comments = post.comments

In [158]:
from nltk import word_tokenize, sent_tokenize
# nltk.download()
from bs4 import BeautifulSoup as soup
from nltk.sentiment import SentimentIntensityAnalyzer
from colour import Color

# (Inadequate) sentiment analysis
def sentiment(reply):
	sents = sent_tokenize(reply)[0]
	SIA = SentimentIntensityAnalyzer()
	return SIA.polarity_scores(sents)["compound"]

def clean(raw):
	parsed = soup(raw)

	# Remove all <blockquotes>
	for b in parsed.find_all('blockquote'):
		b.extract()
	
	return parsed.get_text()

lo, hi = Color("#ffffff"), Color("#ffea80")
gradient = list(lo.range_to(hi, 101))

In [167]:
pruned = [0, 0, 0]
largest = 0
pairs = []

def process(parent, comment):
    cur = {
        "content": clean(comment.body_html),
        "id": comment.id,
        "name": sha1(f"{comment.author.name}".encode()).hexdigest(),
        "depth": comment.depth+1,
        "votes": 0 if comment.score_hidden else abs(comment.score),
        "ranked": True if comment.author_flair_text else False,
        "spicy": bool(comment.controversiality),
        "score": 0,
        "threadScore": 0,
        "below": False, # max # layers below
        "agree": None,
        "replies": []
    }
    
    # Prune
    if not cur["spicy"] and cur["votes"] < 3:
        pruned[0] += 1; return
    # if len(cur["content"]) < 50:
    wordCount = len(word_tokenize(cur["content"]))
    if wordCount < 15: pruned[1] += 1; return

    maxBelow = 0
    for r in comment.replies:
        if isinstance(r, more) or not r.author: continue
        replyThread = process(cur, r)  # Recurse
        if replyThread: 
            cur["replies"].append(replyThread)
            cur["threadScore"] += replyThread["score"]
            if replyThread["below"] is not False:
                if replyThread["below"] > maxBelow: maxBelow = replyThread["below"]

    # Depth/below calc
    if not cur["replies"]:
        cur["below"] = 0 # False -> 0: hit a leaf
        if cur["depth"] < 3: 
            pruned[2] += 1; return
    else:
        cur["below"] = maxBelow + 1

    # Score calculation
    S = 1.2 if cur["spicy"] else 1   # boost controvserial comment scores
    R = 1.4 if cur["ranked"] else 1  # favor "seasoned" commenters
    L = sqrt((wordCount-10)/5)       # encourage longer comments
    D = cur["depth"]                 # boost deep comment scores
    B = (cur["below"]+.5)/2          # devalue no/shallow subthreads

    cur["score"] = S*R*L*D*B*cur["votes"]
    cur["threadScore"] += cur["score"]
    global largest
    if cur["threadScore"] > largest: largest = cur["threadScore"]

    # (Dis)agreement
    # sen = sentiment(cur["content"])
    # cur["agree"] = False if sen < -0.7 else (True if sen > 0.9 else None)

    # Save pairs for classification en masse
    pairs.append({
        "id": cur["id"],
        "pre_text": topic if not parent else parent["content"], 
        # "pre_text": topic if not parent else ''.join(sent_tokenize(parent["content"])[:2]),
        "con_text": cur["content"],
        # "con_text": ''.join(sent_tokenize(cur["content"])[:2]),
    })

    return cur 


def traverse(comments):
    threads = []
    for c in comments: # Each TLC
        if isinstance(c, more) or c.stickied: continue
        if c.author is None: continue
        thread = process({}, c)
        if thread: 
            threads.append(thread)
            # print(f"{thread['threadScore']:0.2f}")
    return threads

tree = {
    "root": {
        "id": ID,
        "content": topic,
        "replies": traverse(comments)
    }
}


In [160]:
cfg = {
    "blend_warmup": True,
    "blend_type": "no",
    "blend_rate": [0] * 100,
    "train_blend_warmup": False,
    "n_epochs_blend_only": 5,

    "debug_mode": False, #True,

    "device": "cuda",
    "pt_model": "bert-base-uncased",
    "n_trials": 1,
    "n_epochs": 50,
    "learn_rate": 1e-5,
    "epsilon": 1e-8,
    "pivot_metric": "auc",
    "early_stop": 3,
    "tasks": {
        "main": {
            "classes": ["sup", "att", "neu"] 
        },
        "nli": {
            "data_paths": ["data/rsc/mnli-org.csv",
                           "data/rsc/antsyn-nli.csv"],
            "classes": ["ent", "con", "neu"],
        },
        "senti": {
            "data_paths": ["data/rsc/senti-irish.csv",
                           "data/rsc/senti-ldong.csv",
                           "data/rsc/senti-mm.csv",
                           "data/rsc/senti-sem17.csv",
                           "data/rsc/senti-norm.csv"],
            "classes": ["pos", "neg", "neu"],
        },
        "causal": {
            "data_paths": ["data/rsc/because-causal.csv",
                           "data/rsc/conet-causal.csv",
                           "data/rsc/pdtb-i-causal.csv",
                           "data/rsc/wiqa-causal.csv"],
            "classes": ["cause", "obstruct", "precede", "sync", "else"],
        },
        "normarg_polar": {
            "data_path": "data/rsc/normarg.csv",
            "classes": ["consist", "contrast"],
        },
        "normarg_jtype": {
            "data_path": "data/rsc/normarg.csv",
            "classes": ["norm", "conseq"],
        },
        "normarg_senti": {
            "data_path": "data/rsc/normarg.csv",
            "classes": ["positive", "negative"],
            "con_classes": ["advocate", "object"],
        },
 
    },
    "max_n_batch_tokens": 512*5,
    "max_batch_size": 6,
    "data_dir": None,
    "logs_dir": None,
    "save_model": False,
    "models_dir": "models",
    "rel2label": {
            "1": 0,  # support
            "-1": 1,  # attack
            "0": 2  # neutral
    }
}

tasks = ["main"] + ["nli", "senti", "normarg"]


cfg["task2classes"] = {}
for task in tasks:
    if task == "normarg":
        cfg["task2classes"]["normarg_polar"] = cfg["tasks"]["normarg_polar"]["classes"]
        cfg["task2classes"]["normarg_jtype"] = cfg["tasks"]["normarg_jtype"]["classes"]
        cfg["task2classes"]["normarg_norm_senti"] = cfg["tasks"]["normarg_senti"]["classes"]
        cfg["task2classes"]["normarg_conseq_senti"] = cfg["tasks"]["normarg_senti"]["classes"]
    else:
        cfg["task2classes"][task] = cfg["tasks"][task]["classes"]

In [168]:
from csv_utils import *
from models import *
import os

MODEL = 'trained.model'

assert os.path.exists(MODEL), f"Model not found: {MODEL}"
# Prepare the trainer
trainer = Trainer(cfg)

# Warmup with blend tasks
init_model = torch.load(MODEL, map_location='cuda')
trainer.init_model(init_model)
trainer.model = trainer.model.to('cuda')

# Prediction
keys = ["id", "con_text", "pre_text", "label_prob", "label_pred"]

print(f"Loading data...")
batches = trainer.make_batches(pairs, ["pre", "con"])

print(f"Classifying {sum(len(b["id"]) for b in batches)} replies over {len(batches)} batches:")

trainer.run_epoch(batches, "predict", "main")

# for batch in batches:
#     for i, id in enumerate(batch["id"]):
#         print(batch["pre_text"][i][:20],'\t',batch["con_text"][i][:20],'\t',batch["label_prob"][i],'\t',batch["label_pred"][i])
res = {ID: [b["label_pred"][i], b["label_prob"][i]] for b in batches for i, ID in enumerate(b["id"])}
# for ID in res:
#     print(f"{ID}: {res[ID]}")

Loading pretrained model...
Loading data...
Classifying 92 replies over 17 batches:


100%|██████████| 17/17 [00:00<00:00, 18.40it/s]


In [174]:
vis = Digraph(format='png')

def build(parID, cur):
    # print(res[cur["id"]])
    g = int(25*(cur["threadScore"]/largest*100)**(.25)+0)
    
    pred = res[cur["id"]][0]; prob = res[cur["id"]][1]
    prob = [round(x,3) for x in prob]
    label = f"{cur['threadScore']:.2f} ({cur['score']:.2f})\n{prob}\n{textwrap.fill(cur["content"], 25)[:200]}{'...' if len(cur["content"]) > 200 else ''}"
    vis.node(cur["id"], label=label, shape='box', style='filled', fillcolor=gradient[g].hex_l)

    # cur["agree"] = None if pred == 2 else not bool(pred)
    cur["agree"] = True if (prob[0] > prob[1]) else False
    edge_color = 'black' if cur["agree"] is None else ('green' if cur["agree"] else 'red')
    edge_label = '?' if cur["agree"] is None else ('Agree' if cur["agree"] else 'Disagree')

    vis.edge(parID, cur["id"], label=edge_label, color=edge_color, fontcolor=edge_color)

    # Recurse for the replies
    for r in cur["replies"]:
        build(cur["id"], r)

# Add a root node to the graph with the topic
root = tree["root"]
vis.node(root["id"], label=topic, shape='box')

for r in root['replies']:
    build(root["id"], r)

In [175]:
pp = pprint.PrettyPrinter(indent=4, sort_dicts=False)
# pp.pprint(tree)
json.dump(tree, open("tree.json", "w"), indent=4, sort_keys=False)

print(
	"Pruned comments:\n"
	f"\t{pruned[0]} with low engagement\n"
	f"\t{pruned[1]} were too short\n"
	f"\t{pruned[2]} from shallow threads\n"
)

vis.render('tree', view=True)

Pruned comments:
	42 with low engagement
	27 were too short
	47 from shallow threads



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'tree.png'

F: Not sharing "/usr/share/icons" with sandbox: Path "/usr" is reserved by Flatpak

(loupe:2): GLib-GObject-CRITICAL **: 21:19:22.399: g_object_weak_unref: couldn't find weak ref 0x7ff5c8352fb0((nil))

(loupe:2): GLib-GObject-CRITICAL **: 21:19:43.768: g_object_weak_unref: couldn't find weak ref 0x7ff5c8352fb0((nil))


In [176]:
script = tree
vis2 = Digraph(format='png')

def build(parID, cur):
    if cur["agree"]: return
    cur["replies"].sort(key=lambda x: x["threadScore"], reverse=True)
    g = int(25*(cur["threadScore"]/largest*100)**(.25)+0)
    
    label = f"{textwrap.fill(cur["content"], 50)[:1000]}{'...' if len(cur["content"]) > 1000 else ''}"
    vis2.node(cur["id"], label=label, shape='box', style='filled', fillcolor=gradient[g].hex_l)

    edge_color = 'black' if cur["agree"] is None else ('green' if cur["agree"] else 'red')
    edge_label = '?' if cur["agree"] is None else ('Agree' if cur["agree"] else 'Disagree')

    vis2.edge(parID, cur["id"], label=edge_label, color=edge_color, fontcolor=edge_color)

    # Recurse for the replies
    if cur["replies"]:
        build(cur["id"], cur["replies"][0])


vis2.node(root["id"], label=topic, shape='box')
root = script["root"]
root["replies"].sort(key=lambda x: x["threadScore"], reverse=True)
for tlc in root["replies"][:5]: # explore top 3 threads
    build(root["id"], tlc)
    print(tlc["threadScore"])


json.dump(script, open("script.json", "w"), indent=4, sort_keys=False)

vis2.render('script', view=True)


4332.052884826169
3474.562500742937
847.1289219104086
733.8629874438277
635.6679389227189


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'script.png'

F: Not sharing "/usr/share/icons" with sandbox: Path "/usr" is reserved by Flatpak

(loupe:2): Gtk-WARNING **: 21:20:57.574: Trying to snapshot GtkGizmo 0x55c98c72fff0 without a current allocation

(loupe:2): Gtk-WARNING **: 21:36:46.717: Trying to snapshot GtkGizmo 0x55c98c4e3210 without a current allocation
